In [1]:
from datasets import load_dataset
from datasets.arrow_dataset import Dataset
from datasets.dataset_dict import DatasetDict, IterableDatasetDict
from datasets.iterable_dataset import IterableDataset
import torch

import dataHandler as dh

In [2]:
split_dataset, test_dict = dh.getMultiToxigenDataset(['black', 'asian', 'women'], is_random=True, test_case=True)

In [3]:
from transformers import AutoTokenizer
 
# Model id to load the tokenizer
model_id = "answerdotai/ModernBERT-base"

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
 
# Tokenize helper function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, return_tensors="pt", max_length=140)
 
# Tokenize dataset
if "label" in split_dataset["train"].features.keys():
    split_dataset =  split_dataset.rename_column("label", "labels") # to match Trainer
tokenized_dataset = split_dataset.map(tokenize, batched=True, remove_columns=["text"])
 
tokenized_dataset["train"].features.keys()
# dict_keys(['labels', 'input_ids', 'attention_mask'])


Map:   0%|          | 0/44716 [00:00<?, ? examples/s]

Map:   0%|          | 0/11179 [00:00<?, ? examples/s]

dict_keys(['labels', 'input_ids', 'attention_mask'])

In [4]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 44716
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 11179
    })
})

In [5]:
from transformers import AutoModelForSequenceClassification
 
# Model id to load the tokenizer
model_id = "answerdotai/ModernBERT-base"
 
# Prepare model labels - useful for inference
labels = ["no hate", "hate"]
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
 
# Download the model from huggingface.co/models
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label,
)


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

ModernBertForSequenceClassification(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(50368, 768, padding_idx=50283)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertRotaryEmbedding()
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=2304, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=1152, out_features=768, bias=False)
        )
      

In [7]:
torch.cuda.is_available()

True

In [8]:
import numpy as np
from sklearn.metrics import f1_score
 
# Metric helper method
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    score = f1_score(
            labels, predictions, labels=labels, pos_label=1, average="weighted"
        )
    return {"f1": float(score) if score == 1 else score}


In [9]:
from huggingface_hub import HfFolder
from transformers import Trainer, TrainingArguments
 
# Define training args
training_args = TrainingArguments(
    output_dir= "ModernBERT-domain-classifier",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    num_train_epochs=3,
    bf16=True, # bfloat16 training 
    optim="adamw_torch_fused", # improved optimizer 
    # logging & evaluation strategies
    logging_strategy="steps",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
    # push to hub parameters
    # push_to_hub=True,
    # hub_strategy="every_save",
    # hub_token=HfFolder.get_token(),
)
 
# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)
trainer.train()
# {'train_runtime': 3642.7783, 'train_samples_per_second': 1.235, 'train_steps_per_second': 0.04, 'train_loss': 0.535627057634551, 'epoch': 5.0}


Epoch,Training Loss,Validation Loss,F1
1,0.208100,0.203152,0.934372
2,0.078600,0.264002,0.936627
3,0.009400,0.516143,0.936559


TrainOutput(global_step=4194, training_loss=0.1120114601761259, metrics={'train_runtime': 672.0376, 'train_samples_per_second': 199.614, 'train_steps_per_second': 6.241, 'total_flos': 1.249937049576096e+16, 'train_loss': 0.1120114601761259, 'epoch': 3.0})

In [10]:
model.save_pretrained("ModernBERT-domain-classifier_local")
tokenizer.save_pretrained("ModernBERT-domain-classifier_local_tokenizer")

('ModernBERT-domain-classifier_local_tokenizer\\tokenizer_config.json',
 'ModernBERT-domain-classifier_local_tokenizer\\special_tokens_map.json',
 'ModernBERT-domain-classifier_local_tokenizer\\tokenizer.json')

## Creating the tests from a Pipeline

In [11]:
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
 
# load model from huggingface.co/models using our repository id
classifier = pipeline(
    task="text-classification", 
    tokenizer = AutoTokenizer.from_pretrained("ModernBERT-domain-classifier_local_tokenizer"),
    model=AutoModelForSequenceClassification.from_pretrained("ModernBERT-domain-classifier_local"),
    device=0 if torch.cuda.is_available() else -1
)

Device set to use cuda:0


In [12]:
from checklist.pred_wrapper import PredictorWrapper
from checklist.test_types import MFT

class HuggingFacePipelineWrapper(PredictorWrapper):
    def __init__(self, pipeline_classifier):
        self.pipeline = pipeline_classifier
        
    def __call__(self, texts):
        predictions = []
        confidences = []
        
        for text in texts:
            prediction = self.pipeline(text)
            
            # Adjust based on your pipeline's output format
            prediction = prediction[0]  # Get first result if multiple
            
            
            label = prediction['label']
            score = prediction['score']
                
            predictions.append(label)
            confidences.append(score)
            
        return predictions, confidences

In [20]:
wrapped_classifier = HuggingFacePipelineWrapper(classifier)

hate_test = MFT(data = test_dict['text'], labels=test_dict['label'], name="Basic Hate Test")

sentiment_results = hate_test.run(wrapped_classifier, overwrite=True)

Predicting 2942 examples


In [21]:
hate_test.summary()

Test cases:      2942
Fails (rate):    250 (8.5%)

Example fails:
no hate  no, you can't wear headscarves and hijabs, that's cultural appropriation. no, armpit hair is normal, shaving it off
----
no hate  women have been little more than a servant since time began
----
no hate asian people are the best -*- asian people aren't the best *- my dad married an asian lady -*- my dad
----
